In [12]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_csv("../../data/Data_Repuestos_2018_2021_v2.csv",
                       usecols=[' noPeriodo',' idArticulo','MoImporte',' moPrecio', 'CnSalida 2018', ' caDescProduct', 'Factura o Boleta'],
                       )
product = pd.read_csv("../../data/producto.csv")
df.head()

,noPeriodo,idArticulo,CnSalida 2018,moPrecio,MoImporte,Factura o Boleta,caDescProduct
0,20180100,25319,1.0,15.00,15.0,2018-1-159166- BOL,FOCO H11-12 100-90W CURVO
1,20180100,4923,1.0,23.00,23.0,2018-1-159167- BOL,FILTRO AIRE HYUNDAI TUCSON
2,20180100,25853,1.0,13.00,13.0,2018-1-159169- BOL,FILTRO GASOLINA NISSAN METAL GA15-16 INJECTADO
3,20180100,13497,1.0,90.00,90.0,2018-1-159171- BOL,JGO CABLES BUJIA KIA RIO LARGO
4,20180100,23900,4.0,6.25,25.0,2018-1-159171- BOL,BUJIA NGK


In [13]:
df.columns = [colum.strip() for colum in df.columns.tolist()]
product.columns = [colum.strip() for colum in product.columns.tolist()]

### Seleccionando solo los productos más relevantes

In [14]:
products = product['idArticulo'].unique().tolist()

In [15]:
df = df.loc[df['idArticulo'].isin(products)]
#df['Factura o Boleta'] = df['Factura o Boleta'].astype('category')
df['caDescProduct'] = df['caDescProduct'].str.strip()
#df['caDescProduct'] = df['caDescProduct'].astype('category')
df.dropna(axis=0, subset=['Factura o Boleta'], inplace=True)
df['Factura o Boleta'] = df['Factura o Boleta'].astype('str')
df.head()

,noPeriodo,idArticulo,CnSalida 2018,moPrecio,MoImporte,Factura o Boleta,caDescProduct
4,20180100,23900,4.0,6.25,25.00,2018-1-159171- BOL,BUJIA NGK
11,20180100,9950,1.0,10.00,10.00,2018-1-159176- BOL,REFRIGERANTE COOLANT AZUL/NATURAL TRANSP. GALO...
17,20180100,23900,4.0,7.00,28.00,2018-1-159179- BOL,BUJIA NGK
43,20180100,11397,4.0,7.00,28.00,2018-1-61508- FAC,BUJIA NGK 5/8 1/E R/L E/P 6953
52,20180100,23900,5.0,6.25,31.25,2018-1-61511- FAC,BUJIA NGK


In [16]:
basket = (df[['Factura o Boleta', 'idArticulo', 'CnSalida 2018']]
          .groupby(['Factura o Boleta', 'idArticulo'])['CnSalida 2018']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Factura o Boleta'))
basket.head()

idArticulo,1627,3882,3883,4827,9776,9950,10702,10960,11395,11397,11509,11807,16404,17779,17804,20301,23900
Factura o Boleta,,,,,,,,,,,,,,,,,
2018- BB01-1017- BOL,0.0,0.0,0.0,0.0,242.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1037- BOL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1042- BOL,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1044- BOL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018- BB01-1046- BOL,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    return 0

basket_sets = basket.applymap(encode_units)


## Generando conjuntos de articulos frecuentes entre si [Link](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#:~:text=a%20frequent%20itemset%20is%20defined%20as%20a%20set%20of%20items%20that%20occur%20together%20in%20at%20least%2050%25%20of%20all%20transactions%20in%20the%20database.)


* Estamos usando un soporte del 7% de sensibilidad

In [19]:
frequent_itemsets = apriori(basket_sets, min_support=0.002, use_colnames=True)

In [20]:
frequent_itemsets.head()

,support,itemsets
0,0.057805,(1627)
1,0.027248,(3882)
2,0.037953,(3883)
3,0.041261,(4827)
4,0.080576,(9776)


## Generamos las reglas con su respectivo soporte, relación y relevancia. [link](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#:~:text=support%20is%20used%20to%20measure%20the%20abundance%20or%20frequency%20(often%20interpreted%20as%20significance%20or%20importance)%20of%20an%20itemset%20in%20a%20database.)

* Por qué la métrica **lift** ?[link](https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/#:~:text=The%20lift%20metric%20is%20commonly%20used%20to%20measure%20how%20much%20more%20often%20the%20antecedent%20and%20consequent%20of%20a%20rule%20A-%3EC%20occur%20together%20than%20we%20would%20expect%20if%20they%20were%20statistically%20independent.%20If%20A%20and%20C%20are%20independent%2C%20the%20Lift%20score%20will%20be%20exactly%201.)
* 1 quiere decir que son productos independientes, superior a 1 que contienen cierta relación

In [21]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(3882),(9950),0.027248,0.124173,0.004087,0.150000,1.207994,0.000704,1.030385
1,(9950),(3882),0.124173,0.027248,0.004087,0.032915,1.207994,0.000704,1.005860
2,(4827),(11509),0.041261,0.038536,0.002725,0.066038,1.713646,0.001135,1.029446
3,(11509),(4827),0.038536,0.041261,0.002725,0.070707,1.713646,0.001135,1.031686
4,(4827),(17804),0.041261,0.037174,0.002336,0.056604,1.522671,0.000802,1.020596


In [22]:
rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         8 non-null      object 
 1   consequents         8 non-null      object 
 2   antecedent support  8 non-null      float64
 3   consequent support  8 non-null      float64
 4   support             8 non-null      float64
 5   confidence          8 non-null      float64
 6   lift                8 non-null      float64
 7   leverage            8 non-null      float64
 8   conviction          8 non-null      float64
dtypes: float64(7), object(2)
memory usage: 704.0+ bytes


In [23]:
results = pd.DataFrame.from_dict({"antecedents": [list(item)[0] for item in rules.antecedents.tolist()],
                                  "consequents":[list(item)[0] for item in rules.consequents.tolist()]})

In [24]:
results.head()

,antecedents,consequents
0,3882,9950
1,9950,3882
2,4827,11509
3,11509,4827
4,4827,17804


In [28]:
results.rename(columns={
    "consequents": "idArticulo"
    }, 
    inplace=True)

In [29]:
results.to_csv('../../data/basket_analysis.csv', index=False)